In [ ]:
import random
import numpy as np
import math
import sys
import requests
import io

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt


matplotlib.rcParams['figure.figsize'] = (8, 7)
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['legend.fontsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['lines.linewidth'] = 2

random.seed(42)
np.random.seed(42)

# Load data from github

The dataset contains simulated and reconstructed jets, ie. collimated sprays of particles, as seen in particle detectors at the CERN LHC. For each jet the following features have been saved:
* jet kinematic properties: transverse momentum ($p_T$), pseudorapidity ($\eta$) and mass ("sdmass")
* number of clustered particles, ie. constituents ("nparticles")
* substructure variables, ie. n-subjetiness, $\tau_{N}$
* jet charge sum from consituents

The jets are preselected to originate from one of the following decays:
* Z boson decay: $Z\to q\bar{q}$
* W boson decay: $W\to q\bar{q}^{\prime}$
* Top quark decay: $t\to bW, W\to q\bar{q}^{\prime}$

In [ ]:
response = requests.get('https://raw.githubusercontent.com/matt-komm/MLPrimer/main/data.npz')
response.raise_for_status()
data = np.load(io.BytesIO(response.content))

Extract feature arrays from loaded data. Stack features into single array of dimension `[<#samples>,<#features>]` for ML training.

In [ ]:
feature_names = [
    'jet_pt','jet_eta','jet_nparticles','jet_sdmass',
    'jet_tau1','jet_tau2','jet_tau3','jet_tau4',
    'jet_charge'
]

jet_features = np.stack([data[name] for name in feature_names], axis=1)

jet_imgs = data['jet_img'][:,:,:,0:1]

label_names = ['Wqq','Zqq','Tbqq']
jet_labels = np.sum([i*(data['label_'+name]>0) for i,name in enumerate(label_names)],axis=0)

print ('Jet features: ',jet_features.shape)
print ('Jet images: ',jet_imgs.shape)
print ('Jet labels: ',jet_labels.shape)


# Plot feature distributions

In [ ]:
plt.figure(figsize=[12,12])
for i,feature in enumerate(feature_names):
    plt.subplot(3, 3, i+1)
    _,bins,_ = plt.hist(data[feature][data['label_Tbqq']>0],bins=25,alpha=0.5,label='Wqq')
    plt.hist(data[feature][data['label_Zqq']>0],bins=bins,alpha=0.5,label='Zqq')
    plt.hist(data[feature][data['label_Wqq']>0],bins=bins,alpha=0.5,label='Tbqq')
    plt.xlabel(feature)
    plt.legend()
plt.show()

# Split randomly into train and test samples

In [ ]:
import sklearn.model_selection

jet_feature_train, jet_feature_test, jet_img_train, jet_img_test, label_train, label_test = \
    sklearn.model_selection.train_test_split(jet_features, jet_imgs, jet_labels, test_size=0.33, random_state=42)

print ('Split jet features: ',jet_feature_train.shape,jet_feature_test.shape)
print ('Split jet images: ',jet_img_train.shape,jet_img_test.shape)
print ('Split jet labels: ',label_train.shape,label_test.shape)


# Training a BDT

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

bdt = HistGradientBoostingClassifier(
    max_iter=100,
    learning_rate=0.2,
    max_depth=8,
    #max_depth=3,
    #min_samples_leaf=2000,
    random_state=42,
    verbose=1,
    early_stopping=False,
)
bdt.fit(jet_feature_train, label_train)

# Plot loss vs iteration

In [ ]:
bdt_scores_train = bdt.predict_proba(jet_feature_train)
bdt_scores_test = bdt.predict_proba(jet_feature_test)

def get_loss_vs_iteration(x_features, y_labels, classifier):
    return np.array(list(map(
        lambda score: sklearn.metrics.log_loss(y_labels,score), classifier.staged_predict_proba(x_features)
    )))

train_loss_bdt = get_loss_vs_iteration(jet_feature_train, label_train, bdt)
test_loss_bdt = get_loss_vs_iteration(jet_feature_test, label_test, bdt)

plt.figure()
plt.plot(np.arange(len(train_loss_bdt)),train_loss_bdt,label="BDT (train)",color='royalblue',linestyle='-')
plt.plot(np.arange(len(test_loss_bdt)),test_loss_bdt,label="BDT (test)",color='blue',linestyle='--')

plt.ylabel("log(loss)")
plt.xlabel("Iteration")
plt.legend()
plt.grid()
plt.show()
plt.close()


# Plot BDT output scores

In [ ]:
plt.figure(figsize=[13,4])
for ilabel in range(3):
    plt.subplot(1, 3, ilabel+1)
    _,bins,_ = plt.hist(bdt_scores_train[label_train==0,ilabel],bins=25,alpha=0.5,label='Wqq',color='royalblue',density=True)
    plt.hist(bdt_scores_train[label_train==1,ilabel],bins=bins,alpha=0.5,label='Zqq',color='limegreen',density=True)
    plt.hist(bdt_scores_train[label_train==2,ilabel],bins=bins,alpha=0.5,label='Tbqq',color='darkorange',density=True)

    plt.hist(bdt_scores_test[label_test==0,ilabel],bins=bins,color='blue',histtype='step',density=True)
    plt.hist(bdt_scores_test[label_test==1,ilabel],bins=bins,histtype='step',color='darkgreen',density=True)
    plt.hist(bdt_scores_test[label_test==2,ilabel],bins=bins,histtype='step',color='orangered',density=True)

    plt.xlabel("BDT score "+label_names[ilabel])
    plt.legend()
plt.show()
plt.close()

# Deep neural network training with Keras+Tensorflow

In [ ]:
import tensorflow as tf

# Building the network

In [ ]:
tf.random.set_seed(42)
tf.keras.utils.set_random_seed(42)

inputLayer = tf.keras.Input(shape=jet_features.shape[1:])
nnLayer = inputLayer
for nodes in [64,64,64]:
    nnLayer = tf.keras.layers.Dense(nodes, activation='selu')(nnLayer)
    nnLayer = tf.keras.layers.Dropout(0.1)(nnLayer)
outputLayer = tf.keras.layers.Dense(3, activation=None)(nnLayer)

model = tf.keras.models.Model(inputs=[inputLayer],outputs=[outputLayer])
model.summary()

# Configure the training

In [ ]:
model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=2e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train the model

In [ ]:
'''
for feature in ['jet_pt','jet_nparticles','jet_sdmass']:
    idx = feature_names.index(feature)
    jet_feature_train[:,idx] = np.log(1+jet_feature_train[:,idx])
    jet_feature_test[:,idx] = np.log(1+jet_feature_test[:,idx])

scaler = sklearn.preprocessing.StandardScaler()
jet_feature_train = scaler.fit_transform(jet_feature_train)
jet_feature_test = scaler.transform(jet_feature_test)
'''

def step_decay(epoch):
    initial_lrate = 3e-2
    drop = 0.7
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, (1.+epoch)/epochs_drop)
    return lrate
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)


trainProgress = model.fit(
    x=[jet_feature_train],
    y=[tf.keras.utils.to_categorical(label_train, num_classes = 3)],
    batch_size=256,
    epochs=100,
    verbose=1,
    validation_split=0.25,
    shuffle = True,
    callbacks=[lrate]
)

# Compare against BDT




In [ ]:
nn_scores_train = model.predict(jet_feature_train)

#note: NN outputs are logits here; need to apply softmax to get likelihoods
nn_scores_test = tf.nn.softmax(model.predict(jet_feature_test))

plt.figure()
plt.plot(np.arange(len(train_loss_bdt)),train_loss_bdt,label="BDT (train)",color='royalblue',linestyle='-')
plt.plot(np.arange(len(test_loss_bdt)),test_loss_bdt,label="BDT (test)",color='blue',linestyle='--')
plt.plot(np.arange(len(trainProgress.history['loss'])),trainProgress.history['loss'],label="DNN (train)",color='lightcoral',linestyle='-')
plt.plot(np.arange(len(trainProgress.history['val_loss'])),trainProgress.history['val_loss'],label="DNN (test)",color='red',linestyle='--')
plt.ylabel("log(loss)")
plt.xlabel("Iteration/Epoch")
plt.ylim([0.4,1.])
plt.grid()
plt.legend()
plt.show()
plt.close()

# Plot Receiver-operator-characteristic (ROC) curve

In [ ]:
bdt_fpr,bdt_tpr,bdt_thres = sklearn.metrics.roc_curve(label_test==2,bdt_scores_test[:,2])
nn_fpr,nn_tpr,nn_thres = sklearn.metrics.roc_curve(label_test==2,nn_scores_test[:,2])

plt.figure()
plt.plot(bdt_tpr,bdt_fpr,color='royalblue',label='BDT test')
plt.plot(nn_tpr,nn_fpr,color='lightcoral',linestyle='--',label='NN test')
plt.xlabel("Tbqq efficiency")
plt.ylabel("Wqq+Zqq efficiency")
plt.grid()
plt.xlim([0,1])
plt.yscale('log')
plt.ylim([1e-4,1])
plt.legend()
plt.show()
plt.close()

# Jet images

In [ ]:
imageScaler = sklearn.preprocessing.StandardScaler()
jet_img_train = imageScaler.fit_transform(jet_img_train.reshape(jet_img_train.shape[0],-1)).reshape(jet_img_train.shape)
jet_img_test = imageScaler.transform(jet_img_test.reshape(jet_img_test.shape[0],-1)).reshape(jet_img_test.shape)

plt.figure(figsize=[11,4])
plt.subplot(1, 2, 1)
plt.title("Pixel mean")
plt.imshow(imageScaler.mean_.reshape(jet_img_train.shape[1:]), extent=(-0.8, 0.8, -0.8, 0.8), cmap='seismic')
plt.ylabel("$\Delta\eta$")
plt.xlabel("$\Delta\phi$")
plt.colorbar()
plt.subplot(1, 2, 2)
plt.title("Pixel scale")
plt.imshow(imageScaler.scale_.reshape(jet_img_train.shape[1:]), extent=(-0.8, 0.8, -0.8, 0.8), cmap='seismic')
plt.ylabel("$\Delta\eta$")
plt.xlabel("$\Delta\phi$")
plt.colorbar()
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=[15,4])
for ilabel in range(3):
    plt.subplot(1, 3, ilabel+1)
    plt.title(label_names[ilabel])
    plt.imshow(np.mean(jet_img_train[label_train==ilabel],axis=0), extent=(-0.8, 0.8, -0.8, 0.8))#, vmin=-0.25, vmax=0.25, cmap='seismic')
    plt.ylabel("$\Delta\eta$")
    plt.xlabel("$\Delta\phi$")
    plt.colorbar()
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
tf.random.set_seed(42)
tf.keras.utils.set_random_seed(42)

inputFeatureLayer = tf.keras.Input(shape=jet_features.shape[1:])
inputImageLayer = tf.keras.Input(shape=jet_imgs.shape[1:])

convLayer = inputImageLayer
for filter,kernel in [(32,4),(16,2),(16,2),(16,2)]:
    convLayer = tf.keras.layers.Conv2D(filter,kernel, activation='selu', padding='same')(convLayer)
    convLayer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same')(convLayer)
    convLayer = tf.keras.layers.Dropout(0.3)(convLayer)
convLayer = tf.keras.layers.Flatten()(convLayer)

nnLayer = tf.keras.layers.Concatenate()([convLayer,inputFeatureLayer])
for nodes in [64,64,64]:
    nnLayer = tf.keras.layers.Dense(nodes, activation='selu')(nnLayer)
    nnLayer = tf.keras.layers.Dropout(0.1)(nnLayer)
outputLayer = tf.keras.layers.Dense(3, activation=None)(nnLayer)

convModel = tf.keras.models.Model(inputs=[inputFeatureLayer,inputImageLayer],outputs=[outputLayer])
convModel.summary()

In [ ]:
convModel.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=2e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
trainProgressConv = convModel.fit(
    x=[jet_feature_train, jet_img_train],
    y=[tf.keras.utils.to_categorical(label_train, num_classes = 3)],
    batch_size=256,
    epochs=100,
    verbose=1,
    validation_split=0.25,
    shuffle = True,
    callbacks=[lrate]
)

In [ ]:
plt.figure()
plt.plot(np.arange(len(train_loss_bdt)),train_loss_bdt,label="BDT (train)",color='royalblue',linestyle='-')
plt.plot(np.arange(len(test_loss_bdt)),test_loss_bdt,label="BDT (test)",color='blue',linestyle='--')
plt.plot(np.arange(len(trainProgress.history['loss'])),trainProgress.history['loss'],label="DNN (train)",color='lightcoral',linestyle='-')
plt.plot(np.arange(len(trainProgress.history['val_loss'])),trainProgress.history['val_loss'],label="DNN (test)",color='red',linestyle='--')
plt.plot(np.arange(len(trainProgressConv.history['loss'])),trainProgressConv.history['loss'],label="Conv (train)",color='forestgreen',linestyle='-')
plt.plot(np.arange(len(trainProgressConv.history['val_loss'])),trainProgressConv.history['val_loss'],label="Conv (test)",color='darkgreen',linestyle='--')
plt.ylabel("log(loss)")
plt.xlabel("Iteration/Epoch")
plt.ylim([0.4,1.])
plt.grid()
plt.legend()
plt.show()
plt.close()

In [ ]:
conv_scores_test = tf.nn.softmax(convModel.predict([jet_feature_test,jet_img_test]))

In [ ]:
bdt_fpr,bdt_tpr,bdt_thres = sklearn.metrics.roc_curve(label_test==2,bdt_scores_test[:,2])
nn_fpr,nn_tpr,nn_thres = sklearn.metrics.roc_curve(label_test==2,nn_scores_test[:,2])
conv_fpr,conv_tpr,conv_thres = sklearn.metrics.roc_curve(label_test==2,conv_scores_test[:,2])

plt.figure()
plt.plot(bdt_tpr,bdt_fpr,color='royalblue',label='BDT test')
plt.plot(nn_tpr,nn_fpr,color='lightcoral',linestyle='--',label='NN test')
plt.plot(conv_tpr,conv_fpr,color='forestgreen',linestyle='--',label='Conv test')
plt.xlabel("Tbqq efficiency")
plt.ylabel("Wqq+Zqq efficiency")
plt.grid()
plt.xlim([0,1])
plt.yscale('log')
plt.ylim([1e-4,1])
plt.legend()
plt.show()
plt.close()

# Takeaway

* BDTs have far less parameters to configure compared to DNNs
* Overtraining reduces performance and is dangerous if reusing the training dataset
* NNs are susceptible to widely-fluctuating inputs; preprocessing is crucial!
* For "simple" tasks BDTs & DNNs can achieve similar performance when well-tuned
* NNs can "easily" be extended to handle advanced data formats, eg. images; not possible with BDTs!